# Compute the global score

The aim of this notebook is to demonstrate how to compute the global score to evaluate the performance for a given `AugmentedSimulator`. From now on, we assume all the inputs to compute the score are already available.

#### Import required packages

In [ ]:
import math

## Input results description

The result provided below are made-up in order to explain how the score is computed

In [ ]:
allmetrics={"ML":{
                 "x-velocity":0.03,
                 "y-velocity":0.03,
                 "pressure":0.01,
                 "turbulent_viscosity":0.08,
                 "pressure_surfacic":0.27
                 },
            "Physics": 
                 {"spearman_correlation_drag":0.2,
                  "spearman_correlation_lift":0.6,
                  "mean_relative_drag":0.18,
                  "mean_relative_lift":0.25,
                  },
            "OOD": 
                 {
                  "x-velocity":0.08,
                  "y-velocity":0.07,
                  "pressure":0.055,
                  "turbulent_viscosity":0.06,
                  "pressure_surfacic":0.45,
                  "spearman_correlation_drag":0.1,
                  "spearman_correlation_lift":0.6,
                  "mean_relative_drag":0.28,
                  "mean_relative_lift":0.35,
                  }
           }

speedUp= {"ML":1300,"OOD":1300}

We define the acceptability thresholds. 
Each variable is associated with 2 thresholds used to determine whether the result are great, acceptable or unacceptable and whether the result should be maximized or minimized.

In [ ]:
thresholds={"x-velocity":(0.05,0.1,"min"),
            "y-velocity":(0.05,0.1,"min"),
            "pressure":(0.05,0.1,"min"),
            "pressure_surfacic":(0.05,0.1,"min"),
            "turbulent_viscosity":(0.05,0.1,"min"),
            "mean_relative_drag":(0.05,0.1,"min"),
            "mean_relative_lift":(0.05,0.1,"min"),
            "spearman_correlation_drag":(0.5,0.8,"max"),
            "spearman_correlation_lift":(0.5,0.8,"max")           
}

For instance, regarding the value obtained for the variable 'x-velocity'

* if it is lower than 5%, the result is great
* if it is greater than 5% but lower than 10%, the result is acceptable
* if it is greater than 10%, the result is not acceptable

However, for the physical criteria 'spearman_correlation_drag':

* if its value is lower than 0.5, the result is not acceptable
* if it is greater than 0.5 but lower than 0.8, the result is acceptable
* if it is greater than 0.8, the result is great

We also define the configuration

In [ ]:
configuration={
        "coefficients":{"ML":0.4,"OOD":0.3,"Physics":0.3},
        "ratioRelevance":{"Speed-up":0.25,"Accuracy":0.75},
        "valueByColor":{"g":2,"o":1,"r":0},
        "maxSpeedRatioAllowed":10000
    }

We evaluate the result accuracy performances for all variables. We denote by:

* g, a great result
* o, an acceptable result
* r, a not acceptable result

In [ ]:
accuracyResults=dict()
for subcategoryName, subcategoryVal in allmetrics.items():
    accuracyResults[subcategoryName]=[]
    for variableName, variableError in subcategoryVal.items():
        thresholdMin,thresholdMax,evalType=thresholds[variableName]
        if evalType=="min":
            if variableError<thresholdMin:
                accuracyEval="g"
            elif thresholdMin<variableError<thresholdMax:
                accuracyEval="o"
            else:
                accuracyEval="r"
        elif evalType=="max":
            if variableError<thresholdMin:
                accuracyEval="r"
            elif thresholdMin<variableError<thresholdMax:
                accuracyEval="o"
            else:
                accuracyEval="g"

        accuracyResults[subcategoryName].append(accuracyEval)
    
print(accuracyResults)

We are now in position to compute each subscore, with minor prerequisites

In [ ]:
def SpeedMetric(speedUp,speedMax):
    return max(min(math.log10(speedUp)/math.log10(speedMax),1),0)

coefficients=configuration["coefficients"]
ratioRelevance=configuration["ratioRelevance"]
valueByColor=configuration["valueByColor"]
maxSpeedRatioAllowed=configuration["maxSpeedRatioAllowed"]

* ML subscore:

In [ ]:
mlSubscore=0

#Compute accuracy
accuracyMaxPoints=ratioRelevance["Accuracy"]
accuracyResult=sum([valueByColor[color] for color in accuracyResults["ML"]])
accuracyResult=accuracyResult*accuracyMaxPoints/(len(accuracyResults["ML"])*max(valueByColor.values()))
mlSubscore+=accuracyResult

#Compute speed-up
speedUpMaxPoints=ratioRelevance["Speed-up"]
speedUpResult=SpeedMetric(speedUp=speedUp["ML"],speedMax=maxSpeedRatioAllowed)
speedUpResult=speedUpResult*speedUpMaxPoints
mlSubscore+=speedUpResult

* Physics:

In [ ]:
#Compute accuracy
accuracyResult=sum([valueByColor[color] for color in accuracyResults["Physics"]])
accuracyResult=accuracyResult/(len(accuracyResults["Physics"])*max(valueByColor.values()))
physicsSubscore=accuracyResult

* OOD

In [ ]:
oodSubscore=0

#Compute accuracy
accuracyMaxPoints=ratioRelevance["Accuracy"]
accuracyResult=sum([valueByColor[color] for color in accuracyResults["OOD"]])
accuracyResult=accuracyResult*accuracyMaxPoints/(len(accuracyResults["OOD"])*max(valueByColor.values()))
oodSubscore+=accuracyResult

#Compute speed-up
speedUpMaxPoints=ratioRelevance["Speed-up"]
speedUpResult=SpeedMetric(speedUp=speedUp["OOD"],speedMax=maxSpeedRatioAllowed)
speedUpResult=speedUpResult*speedUpMaxPoints
oodSubscore+=speedUpResult

In [ ]:
globalScore=100*(coefficients["ML"]*mlSubscore+coefficients["Physics"]*physicsSubscore+coefficients["OOD"]*oodSubscore)
print(globalScore)